In [ ]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load the Excel file
path="English\English_5.xlsx"

data = pd.read_excel(path, usecols='M')
#data = pd.read_excel(path)
data

In [2]:
text = data['Question Statement (character limit 335)'].str.cat(sep=' ')
text

NameError: name 'data' is not defined

In [ ]:
print(text)

In [ ]:
text = text.replace('\n', '')
text

In [ ]:
len(text)

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp(text)
# Filter out stopwords
text = [token.text for token in doc if not token.is_stop]
# Join tokens back into a string
text=" ".join(text)
print(text)


In [1]:
import re
text = re.sub(r'[^A-Za-z0-9\s]', '', text)
text.strip()

NameError: name 'text' is not defined

In [ ]:
text = re.sub(r'\s+', ' ', text).strip()
text

In [ ]:
type(text)

In [ ]:
len(text)

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
from typing import Optional
class Person(BaseModel):
    """Information of person"""
    name: str  = Field(description="")

In [ ]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of person name about people")

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(model='gpt-4',temperature=0)

In [ ]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": 'Information'})

In [ ]:
extraction_model.invoke(text)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the name of person, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [ ]:
extraction_chain = prompt | extraction_model
extraction_chain

In [ ]:
extraction_chain.invoke({"input": text})

In [ ]:
{
  "people": [
    {"name": "Jatin"},
    {"name": "Reeta"},
    {"name": "Paris"},
    {"name": "Meg"},
    {"name": "Raipur"},
    {"name": "Strawberry"},
    {"name": "Christmas"},
    {"name": "jury"},
    {"name": "grandfather"},
    {"name": "Max"},
    {"name": "Selina"},
    {"name": "Mt Fuji"},
    {"name": "Japan"},
    {"name": "Harry"},
    {"name": "Lion King"},
    {"name": "Mr Park"},
    {"name": "James"},
    {"name": "Ravi"},
    {"name": "Rohan"},
    {"name": "Tripura"},
    {"name": "David"},
    {"name": "Michael"},
    {"name": "Rahul"},
    {"name": "Samsung"},
    {"name": "Apple"},
    {"name": "Lenovo"},
    {"name": "Nuva"},
    {"name": "Elsa"},
    {"name": "Justin Bieber"},
    {"name": "United Nations"},
    {"name": "Dia"},
    {"name": "Ramesh"},
    {"name": "Swift"},
    {"name": "Rohan"}, // Note: "Rohan" appears twice in the list
    {"name": "Bella"},
    {"name": "Tina"},
    {"name": "Blood"},
    {"name": "Assam"},
    {"name": "Information"} // This is added to complete the JSON structure
  ]
}


In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()
extraction_chain.invoke({"input": text})

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")
extraction_chain.invoke({"input": text})